# <center>Applied Data Science - Coursera</center>
## <center>Capstone Project Course</center>
*<center>by Leopoldo Sprandel</center>*


### Business Problem
In May 25th we celebrate the National Wine Day in Brazil. All enthusiasts participates on a two weeks of appreciation, learning sections and winery visits around the country. 
This event promotes the interchange of producers, wine shops and consumers moving all economy around the wine.  
The tourism in the city of São Paulo is in the rout of the event.  
Now imagine we want to choose some Wine Shops in the city to organize a large alliance between them to promote the consumers interchange over the Wine shops.  
The Wine shops need to be well known and we are looking for somehow connections between them.

### Data
The data we choose to select the wine shops that can participate on this alliance are the stores with high recommendations and are part of a network based on the public. 

So, we need to answer two questions:  
- Which wine shops in São Paulo are best evaluated?
- Is the wine shops connected? Which winery shares the same public?  

To answer these questions we can analyze the wine shops listed in the Foursquare API in the city of São Paulo, regarding the following points:
- Check the rank of wine shops (the first 50th)  

| Wine Store | Ranking  |  
|------|------|  
|   WineShopA  | 1|  
|   WineShopB  | 2|  
|   WineShopC  | 3|  
|   WineShopC  | 4|  
|   WineShopD  | ...|  
|   WineShopX  | 50|  

- A network representation can be done connecting stores who shares the same consumer (signalised by the likes).

| Wine Store | Conections  |  
|------|------|  
|   WineShopA  | WineShopB, WineShopC|  
|   WineShopB  | WineShopA, WineShopC, WineShopD, WineShopX|  
|   WineShopC  | ...|   
